In [7]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

Unidades

In [8]:
# deg
deg = np.pi/180

# Solución numérica

La ecuación diferencial es:

$$
z'^2 = \left[ 1 -\frac{1}{2} \left(\frac{z}{R_c}\right)^2 \right]^{-2} - 1 
$$

Planteemos el modelo.

In [9]:
R_c = 1
def model(u, x):
    dudt = np.sqrt((1 - 0.5 * (u/R_c)**2)**(-2) - 1)
    return dudt


Establezcamos las condiciones. En general variaremos $h$. Sin embargo, establezcamos un valor, en este caso, el máximo. 

Además estableceremos -6 como un valor "grande", lejos del cero.

In [10]:
# Condiciones iniciales
h = 1.99  

# Puntos de integración
x = np.linspace(0, -6, 200)  

def z_num(x=x, h=h):
    u = odeint(model, h, x)
    return u


# Solución cercana 

$$
z = h \left( 1 + (R_c/h)^2 \cot^2(\chi) \sin^3(\chi) \left[ \exp( \tan (\chi) \csc^3(\chi) (h/R_c)^2 (x/h) ) - 1 \right] \right)
$$

In [11]:
cot = lambda x: 1/np.tan(x)
csc = lambda x: 1/np.sin(x)


def z_close(x=x, h=h, chi=90*deg):

    return h*(
        1 + (R_c/h)**2 *cot(chi)**2 * np.sin(chi)**3 * (
            np.exp(
                np.tan(chi)*csc(chi)**3 * (h/R_c)**2 * (x/h)
            ) - 1
        )
    )

# Aproximación lineal

$$
z = \cot(\chi) x + h
$$

In [12]:
def z_linear(x=x, chi=90*deg, h=h):
    return cot(chi) * x + h

# Lejos

$$
z = z_0 \exp((x - x_0)/R_c)
$$

In [13]:
def z_far(x=x, x0=0, z0=0):
    return z0*np.exp((x-x0)/R_c)


# Grafica

Podemos hallar $\chi$ con los valores que determinamos numéricamente, de modo que será la pendiente entre los dos primeros puntos.

$$
\chi = \arctan \left( \frac{x_1 - x_0}{z_1 - z_0} \right)
$$

In [14]:
@interact(h=(0.1, 1.99, 0.01))
def graphic(h=1):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    # Graficar los resultados
    numeric = z_num(h=h)

    chi = np.arctan((x[1] - x[0])/(numeric[1] - numeric[0]))
    close = z_close(x, h=h, chi=chi)

    linear = z_linear(x=x, chi=chi, h=h)
    x_num_linear = x[:130]
    linear = linear[:130]


    x0 = x[0]
    z0 = numeric[0]
    far = z_far(x, x0, z0)

    plt.plot(x, numeric, 'b:', label='z numérica', linewidth=2)
    # área bajo la curva
    plt.fill_between(x, numeric.reshape(-1), color='blue', alpha=0.2)
    

    plt.plot(x, close, 'g-', label='z cerca')
    plt.plot(x_num_linear, linear, 'y-', label='z lineal')
    plt.plot(x, far, 'r-', label='z lejos')



    # Ejes del plano cartesiano 
    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    plt.xlabel('Distancia')
    plt.ylabel('z(x)')
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")

    # cuadrícula
    plt.grid(True)

    plt.xlim(-6, 0)
    plt.ylim(-0.3,2.1)
    #plt.axis('equal')

    plt.legend(loc='upper left')

    plt.show()

# graphic(1)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.
